In [1]:
import requests
import base64

import urllib
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
import os

In [2]:
email = 'solujan@gmail.com'
url = f'https://ciberseguridad.diplomatura.unc.edu.ar/cripto/cbc-mac/{email}/challenge'
# Solicitud
request = requests.get(url)
# Body de la respuesta
message = request.content
message

b'from=User <user@example.com>&solujan@gmail.com=1000&comment=Invoice&mac=77d162008b8e3b0cf329d8c7679934db'

In [3]:
mac = message[72:]
mac

b'77d162008b8e3b0cf329d8c7679934db'

In [4]:
clean = b'from=User <user@example.com>&solujan@gmail.com=1000&comment=Invoice'
clean

b'from=User <user@example.com>&solujan@gmail.com=1000&comment=Invoice'

In [5]:
def pad(msg):
  padder = padding.PKCS7(128).padder()
  return padder.update(msg)+padder.finalize()

In [6]:
def create_new_msg(original, prependMessage, originalMac):
  newFirstBlock = bytearray(prependMessage [:16])
  for i in range (16):
    newFirstBlock[i] ^= originalMac[i]
  newFirstBlock = bytes(newFirstBlock)
  return original + newFirstBlock + prependMessage [16:]

originalMessage =  b'from=User <user@example.com>&solujan@gmail.com=1000&comment=Invoice'
prependMessage = b'from=User <user@example.com>&solujan@gmail.com=1000&comment=Invoice'
padded_original_msg = pad(originalMessage)
print(f'padded_original_msg: {padded_original_msg}')
newMessage = create_new_msg(padded_original_msg, prependMessage, mac)
print("Original Message and mac:", originalMessage, mac)
print("New message and mac:", newMessage, mac)


padded_original_msg: b'from=User <user@example.com>&solujan@gmail.com=1000&comment=Invoice\r\r\r\r\r\r\r\r\r\r\r\r\r'
Original Message and mac: b'from=User <user@example.com>&solujan@gmail.com=1000&comment=Invoice' b'77d162008b8e3b0cf329d8c7679934db'
New message and mac: b'from=User <user@example.com>&solujan@gmail.com=1000&comment=Invoice\r\r\r\r\r\r\r\r\r\r\r\r\rQE\x0b\\\x0bgCUJB\x04\x10@\x07B#example.com>&solujan@gmail.com=1000&comment=Invoice' b'77d162008b8e3b0cf329d8c7679934db'


In [ ]:
import urllib.parse

base_url = 'https://ciberseguridad.diplomatura.unc.edu.ar/cripto/cbc-mac/solujan@gmail.com/answer?from=User <user@example.com>&solujan@gmail.com=1000&comment=Invoice\r\r\r\r\r\r\r\r\r\r\r\r\rQE%0b\\%0bgCUJB%04%10@%07B#example.com>&solujan@gmail.com=1000&comment=Invoice&mac=77d162008b8e3b0cf329d8c7679934db'

r = requests.get(base_url)
print(r.url)
print(r.content)

https://ciberseguridad.diplomatura.unc.edu.ar/cripto/cbc-mac/solujan@gmail.com/answer?from=User%20%3Cuser@example.com%3E&solujan@gmail.com=1000&comment=Invoice%0D%0D%0D%0D%0D%0D%0D%0D%0D%0D%0D%0D%0DQE%0B%5C%0BgCUJB%04%10@%07B#example.com%3E&solujan@gmail.com=1000&comment=Invoice&mac=77d162008b8e3b0cf329d8c7679934db
b'Lo siento, siga participando.\n'


# CODIGO DE PRUEBA


In [ ]:
def CBCMAC(message, key, pad=True):
  aesCipher = Cipher(algorithms.AES (key), modes.CBC(bytes(16)), # 16 zero bytes
                     backend=default_backend())
  aesEncryptor = aesCipher.encryptor()
  if pad:
    padder = padding.PKCS7(128).padder()
    padded_message = padder.update(message)+padder.finalize()
  elif len(message) % 16 == 0:
    padded_message = message
  else:
    raise Exception("Unpadded input not a multiple of 16!")

  ciphertext = aesEncryptor.update(padded_message)
  return ciphertext[-16:] # the last 16 bytes are the last block

def pad(msg):
  padder = padding.PKCS7(128).padder()
  return padder.update(msg)+padder.finalize()

def create_new_msg(original, prependMessage, originalMac):
  newFirstBlock = bytearray(prependMessage [:16])
  for i in range (16):
    newFirstBlock[i] ^= originalMac[i]
  newFirstBlock = bytes(newFirstBlock)
  return original + newFirstBlock + prependMessage [16:]

# Creo una key random de 16 bytes
key = os.urandom(32)
# El mensaje a utilizar es el que retorna el servidor
mensaje = b'from=User <user@example.com>&solujan@gmail.com=1000&comment=Invoice'
# Calculo MAC del mensaje a partir de la key
mac1 = CBCMAC(mensaje, key)

print(f'mac 1 : {mac1.hex()} mensaje {mensaje}')
# EXPLOIT: agregao el pad al mensaje y luego creo el nuevo mensaje
pad_msg = pad(mensaje)
mensaje_2 = create_new_msg(pad_msg, mensaje, mac1)

# Calculo el nuevo mac del nuevo mensaje
mac2 = CBCMAC(mensaje_2, key)
print(f'mac 2 : {mac2.hex()} mensaje {mensaje_2}')

if mac1 == mac2:
  print('WINNER')

mac 1 : 14b554f62355f38357e5fce4aded3a6c mensaje b'from=User <user@example.com>&solujan@gmail.com=1000&comment=Invoice'
mac 2 : 14b554f62355f38357e5fce4aded3a6c mensaje b'from=User <user@example.com>&solujan@gmail.com=1000&comment=Invoice\r\r\r\r\r\r\r\r\r\r\r\r\rr\xc7;\x9b\x1e\x00\x80\xe6%\xc5\xc0\x91\xde\x88H,example.com>&solujan@gmail.com=1000&comment=Invoice'
WINNER
